# 目录
## 1. 模块导入
## 2. fashion mnist 数据集准备
## 3. 创建`keras`模型
## 4. `SavedModel`的模型保存、查看、加载、推理
  - 保存模型
  - 查看该模型的所有信息
  - 查看 某版本下的， 签名函数
  - 推理测试  某版本下，某签名服务，做推理
  - 加载模型
  - 查看签名函数
  - 获取签名函数（SavedModel to ConcreteFunction）
  - input Tensor
  - output Tensor
  - 做推理

## 1. 模块导入

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)
    
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. fashion mnist 数据集准备

In [17]:
# 取出fashion mnist 数据集
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()

x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]


# 标准化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

print(x_train_scaled.shape, y_train.shape)
print(x_valid_scaled.shape, y_valid.shape)
print(x_test_scaled.shape, y_test.shape)


# 制作dataset 数据集
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50) # prefetch 先取出50个样本准备
    return dataset


batch_size = 64
epochs = None
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)
valid_dataset = make_dataset(x_valid_scaled, y_valid, 1, batch_size, shuffle=False)
test_dataset = make_dataset(x_test_scaled, y_test, 1, batch_size, shuffle=False)

(55000, 28, 28, 1) (55000,)
(5000, 28, 28, 1) (5000,)
(10000, 28, 28, 1) (10000,)


## 3. 创建`keras`模型

In [3]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=(28, 28, 1)),
        keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=2),

        keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
        keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=2),

        keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
        keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"),
        keras.layers.MaxPool2D(pool_size=2),

        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(lr=0.01), metrics=["accuracy"])
    return model

In [4]:
model = create_model()
model.fit(train_dataset, 
          steps_per_epoch = len(x_train_scaled)//batch_size,
          validation_data=valid_dataset, 
          validation_steps = len(x_valid_scaled)//batch_size,
          epochs=10)

Train for 859 steps, validate for 78 steps
Epoch 1/10
859/859 [==============================] - 7s 8ms/step - loss: 1.0417 - accuracy: 0.6434 - val_loss: 0.6306 - val_accuracy: 0.7622
Epoch 2/10
859/859 [==============================] - 5s 5ms/step - loss: 0.5235 - accuracy: 0.8063 - val_loss: 0.4307 - val_accuracy: 0.8440
Epoch 3/10
859/859 [==============================] - 5s 5ms/step - loss: 0.4280 - accuracy: 0.8427 - val_loss: 0.3708 - val_accuracy: 0.8668
Epoch 4/10
859/859 [==============================] - 5s 5ms/step - loss: 0.3777 - accuracy: 0.8614 - val_loss: 0.3675 - val_accuracy: 0.8664
Epoch 5/10
859/859 [==============================] - 5s 5ms/step - loss: 0.3469 - accuracy: 0.8721 - val_loss: 0.3220 - val_accuracy: 0.8842
Epoch 6/10
859/859 [==============================] - 5s 5ms/step - loss: 0.3244 - accuracy: 0.8805 - val_loss: 0.3208 - val_accuracy: 0.8814
Epoch 7/10
859/859 [==============================] - 5s 5ms/step - loss: 0.3057 - accuracy: 0.8866 - val

## 4. `SavedModel`的模型保存、查看、加载、推理

In [5]:
# 保存模型
tf.saved_model.save(model, "./keras_saved_model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./keras_saved_model/assets


In [6]:
# 查看该模型的所有信息
!saved_model_cli show --dir ./keras_saved_model/ --all

2020-01-31 18:13:34.544723: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-01-31 18:13:34.546340: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['conv2d_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_conv2d_input:0
  The given SavedModel SignatureDef contains the following output

tag-set: 'serve' 是版本信息 <br/>
signature_def['serving_default']  是对外提供推理服务的签名函数

In [7]:
# 查看 某版本下的， 签名函数
!saved_model_cli show --dir ./keras_saved_model/ --tag_set serve --signature_def serving_default

2020-01-31 18:13:36.849147: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-01-31 18:13:36.850965: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
The given SavedModel SignatureDef contains the following input(s):
  inputs['conv2d_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28, 1)
      name: serving_default_conv2d_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [8]:
# 推理测试
# 某版本下，某签名服务，做推理
!saved_model_cli run \
    --dir ./keras_saved_model/ \
    --tag_set serve \
    --signature_def serving_default \
    --input_exprs 'conv2d_input=np.ones((2, 28, 28, 1))'

2020-01-31 18:13:38.858217: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-01-31 18:13:38.859980: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
2020-01-31 18:13:39.620228: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-01-31 18:13:39.623538: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:81:00.0 name: Tesla P100-PCIE-12GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 511.41GiB/s
2020-01-31 18:13:39.623571: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-01-31 18:13:39.623620: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

In [10]:
# 加载模型
load_saved_model = tf.saved_model.load("./keras_saved_model/")

# 查看签名函数
print(load_saved_model.signatures.keys())

KeysView(_SignatureMap({'serving_default': <tensorflow.python.saved_model.load._WrapperFunction object at 0x7f24eadf2b38>}))


In [11]:
# 获取签名函数（SavedModel to ConcreteFunction）
inference = load_saved_model.signatures["serving_default"]
print(inference)  #  类 _WrapperFunction  继承了   ConcreteFunction

In [12]:
# input Tensor
inference.structured_input_signature

((),
 {'conv2d_input': TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='conv2d_input')})

In [13]:
help(inference)

Help on _WrapperFunction in module tensorflow.python.saved_model.load object:

class _WrapperFunction(tensorflow.python.eager.function.ConcreteFunction)
 |  A class wraps a concrete function to handle different distributed contexts.
 |  
 |  The reason for wrapping a concrete function is because the _captured_inputs
 |  fields used for in-replica context and cross-replica context are different.
 |  When `load()` is called from within a tf.distribute.strategy scope, the
 |  captured inputs are distributed variables. When using these distributed
 |  variables during calling the function, we need different approaches when it is
 |  in-replica and when it is not in-replica. When it is in replica, naturally we
 |  should use the corresponding component of the distributed variable; when it is
 |  not in-replica, calling the function should mean that it is constructing a
 |  graph that is not actually going to be used. A typical use case is when
 |  constructing a functional model. In this ca

In [14]:
# output Tensor
inference.structured_outputs

{'dense_1': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_1')}

In [15]:
# 做推理
result = inference(tf.constant(x_test_scaled[:2]))
result

{'dense_1': <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
 array([[1.47661083e-07, 1.18086563e-08, 3.49114856e-07, 8.03321409e-09,
         1.91357822e-06, 1.04567199e-03, 4.27146603e-08, 6.28648093e-03,
         5.68823249e-04, 9.92096543e-01],
        [1.68101964e-04, 1.02938884e-07, 9.74406719e-01, 2.80300742e-06,
         2.31714845e-02, 2.11386055e-06, 2.23859120e-03, 2.44496174e-08,
         9.88680040e-06, 2.43509646e-07]], dtype=float32)>}